In [ ]:
import os,re,h5py
import requests
import zipfile

In [3]:
import numpy as np
import pandas as pd
import geopandas as gpd
import pyproj
import scipy, sys, os, pyproj, glob
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
import pointCollection as pc
import math

# Import some of the scripts that we have written
import sys
#sys.path.append("/home/jovyan/surface_velocity/scripts")
sys.path.append("/home/UOCNT/rag110/Dropbox/UC_files/Data_Wrangling/ICESat2Hackweek2020/surface_velocity/scripts")
from loading_scripts import atl06_to_dict


# run matplotlib in 'widget' mode
%matplotlib widget
%load_ext autoreload
%autoreload 2

In [6]:
#From Ben Smith's code loading in .tif file, running into issues likely with directories
#data_root='/srv/shared/surface_velocity/FIS_Velocity/'
#spatial_extent = np.array([-102, -76, -98, -74.5])
spatial_extent = np.array([-65, -86, -55, -81])
lat=spatial_extent[[1, 3, 3, 1, 1]]
lon=spatial_extent[[2, 2, 0, 0, 2]]
print(lat)
print(lon)
# project the coordinates to Antarctic polar stereographic
xy=np.array(pyproj.Proj(3031)(lon, lat))
# get the bounds of the projected coordinates 
XR=[np.nanmin(xy[0,:]), np.nanmax(xy[0,:])]
YR=[np.nanmin(xy[1,:]), np.nanmax(xy[1,:])]
#Originally tried to load data from a local directory, should change to shared directory
#Measures_vx=pc.grid.data().from_geotif(os.path.join(data_root,'Measures2_FIS_Vx.tif'), bounds=[XR, YR])
#Measures_vy=pc.grid.data().from_geotif(os.path.join(data_root,'Measures2_FIS_Vy.tif'), bounds=[XR, YR])

#Rodrigo Gomez Fell computer path @ UC
data_root = '/mnt/user1/Antarctica/Quantarctica3/Glaciology/MEaSUREs Ice Flow Velocity/'

Measures_vx=pc.grid.data().from_geotif(os.path.join(data_root,'anta_phase_map_VX.tif'), bounds=[XR, YR])
Measures_vy=pc.grid.data().from_geotif(os.path.join(data_root,'anta_phase_map_VY.tif'), bounds=[XR, YR])

[-86 -81 -81 -86 -86]
[-55 -55 -65 -65 -55]


In [15]:
import numpy as np
import pointCollection as pc

def add_surface_velocity_to_is2_dict(is2_dict, spatial_extent, path, vel_x, vel_y ):
    """
    
    is2_dict: Python dictionary with ATL06 track data
    spatial_extent: bounding box of the interest area in the format:
                    (e.g. [-65, -86, -55, -81] == [min_lon, min_lat, max_lon, max_lat])
    path: local path to velocity data
    vel_x: tif velocity raster with x component
    vel_y: tif velocity raster with y component
    
    """
    data_root = path
    
    spatial_extent = np.array([spatial_extent])
    lat=spatial_extent[[1, 3, 3, 1, 1]]
    lon=spatial_extent[[2, 2, 0, 0, 2]]
    print(lat)
    print(lon)
    # project the coordinates to Antarctic polar stereographic
    xy=np.array(pyproj.Proj(3031)(lon, lat))
    # get the bounds of the projected coordinates 
    XR=[np.nanmin(xy[0,:]), np.nanmax(xy[0,:])]
    YR=[np.nanmin(xy[1,:]), np.nanmax(xy[1,:])]
    
    Measures_vx=pc.grid.data().from_geotif(os.path.join(data_root,vel_x), bounds=[XR, YR])
    Measures_vy=pc.grid.data().from_geotif(os.path.join(data_root,vel_y), bounds=[XR, YR])
    
    vx = Measures_vx.interp(is2_dict['x'],is2_dict['y'])
    vy = Measures_vy.interp(is2_dict['x'],is2_dict['y'])

    #Solve for angle to rotate Vy to be along track and Vx to be across track
    import math
    xL=abs((is2_dict['x'][0])-(is2_dict['x'][1]))
    yL=abs((is2_dict['y'][0])-(is2_dict['y'][1]))

    #decides if is descending or ascending path
    if is2_dict['x'][0]-is2_dict['x'][1] < 0:

        theta_rad=math.atan(xL/yL)
        #theta_deg=theta_rad*180/math.pi
        is2_dict['v_along']=vy/math.cos(theta_rad)
        is2_dict['v_across']=vx/math.cos(theta_rad)

    else:
    
        theta_rad=math.atan(xL/yL)
        #theta_deg=theta_rad*180/math.pi
        is2_dict['v_along']=vy/math.sin(theta_rad)
        is2_dict['v_across']=vx/math.sin(theta_rad)
        
    return is2_dict 

In [10]:
data_ATL06='/media/rag110/ADATA SD700/ICESat2/download/'
field_dict={None:['delta_time','latitude','longitude','h_li', 'atl06_quality_summary'],\
                    'ground_track':['x_atc','y_atc'],\
                    'fit_statistics':['dh_fit_dx', 'dh_fit_dy']}

ATL06_files = glob.glob(os.path.join(data_ATL06, 'FIS', f'*ATL06_*_*_003*.h5'))

rgts = {}
for filepath in ATL06_files:
    filename = filepath.split('/')[-1]
    rgt = filename.split('_')[3][0:4]
    track = filename.split('_')[3][4:6]
#     print(rgt,track)
    if not rgt in rgts.keys():
        rgts[rgt] = []
        rgts[rgt].append(track)
    else:
        rgts[rgt].append(track)

In [16]:
### Revised version of code from Ben Smith to read in the hdf5 files and extract necessary datasets and information
def atl06_to_dict(filename, beam, field_dict=None, index=None, epsg=None):
    """
        Read selected datasets from an ATL06 file

        Input arguments:
            filename: ATl06 file to read
            beam: a string specifying which beam is to be read (ex: gt1l, gt1r, gt2l, etc)
            field_dict: A dictinary describing the fields to be read
                    keys give the group names to be read, 
                    entries are lists of datasets within the groups
            index: which entries in each field to read
            epsg: an EPSG code specifying a projection (see www.epsg.org).  Good choices are:
                for Greenland, 3413 (polar stereographic projection, with Greenland along the Y axis)
                for Antarctica, 3031 (polar stereographic projection, centered on the Pouth Pole)
        Output argument:
            D6: dictionary containing ATL06 data.  Each dataset in 
                dataset_dict has its own entry in D6.  Each dataset 
                in D6 contains a numpy array containing the 
                data
    """
    if field_dict is None:
        field_dict={None:['latitude','longitude','h_li', 'atl06_quality_summary'],\
                    'ground_track':['x_atc','y_atc'],\
                    'fit_statistics':['dh_fit_dx', 'dh_fit_dy']}
    D={}
    # below: file_re = regular expression, it will pull apart the regular expression to get the information from the filename
    file_re=re.compile('ATL06_(?P<date>\d+)_(?P<rgt>\d\d\d\d)(?P<cycle>\d\d)(?P<region>\d\d)_(?P<release>\d\d\d)_(?P<version>\d\d).h5')
    with h5py.File(filename,'r') as h5f:
        for key in field_dict:
            for ds in field_dict[key]:
                if key is not None:
                    ds_name=beam+'/land_ice_segments/'+key+'/'+ds
                else:
                    ds_name=beam+'/land_ice_segments/'+ds
                if index is not None:
                    D[ds]=np.array(h5f[ds_name][index])
                else:
                    D[ds]=np.array(h5f[ds_name])
                if '_FillValue' in h5f[ds_name].attrs:
                    bad_vals=D[ds]==h5f[ds_name].attrs['_FillValue']
                    D[ds]=D[ds].astype(float)
                    D[ds][bad_vals]=np.NaN
        D['data_start_utc'] = h5f['/ancillary_data/data_start_utc'][:]
        D['delta_time'] = h5f['/' + beam + '/land_ice_segments/delta_time'][:]
        D['segment_id'] = h5f['/' + beam + '/land_ice_segments/segment_id'][:]
    if epsg is not None:
        xy=np.array(pyproj.proj.Proj(epsg)(D['longitude'], D['latitude']))
        D['x']=xy[0,:].reshape(D['latitude'].shape)
        D['y']=xy[1,:].reshape(D['latitude'].shape)
    temp=file_re.search(filename)
    D['rgt']=int(temp['rgt'])
    D['cycle']=int(temp['cycle'])
    D['beam']=beam
    return D

In [ ]:


beams = ['/gt1r','/gt1l','/gt2r','/gt2l','/gt3r','/gt3l']
D=[]
for file in filename:
    B = ATL06_to_dict(file, dataset_dict=field_dict)
        
    add_surface_velocity_to_is2_dict(B)
    D.append(B)
        